In [2]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.themes import Theme
from bokeh.transform import linear_cmap
from bokeh import palettes
from bokeh.plotting import curdoc
from bokeh.models import HoverTool
import numpy as np
from ipywidgets import interact

output_notebook()

curdoc().theme = 'dark_minimal'

Loading BokehJS ...

# Tight-binding model of 1D chain

## Introduction
This notebook provides a detailed example of the band structure calculation using the tight-binding approximation description of a one-dimensional periodic chain of atoms with two sites per unit cell. This model provides a simplistic but illuminating perspective on electronic structure in solids and offers valuable insight into the behavior of real materials.

The tight-binding model is based on the assumption that the wave function of an electron in a crystal can be represented as a superposition of atomic orbitals centered at each lattice site. In its simplest form, known as the "one-orbital model", it is assumed that only one atomic orbital contributes significantly to the electron's wave function at each site.

In a more advanced "multi-orbital model", such as the one we'll be discussing here, we take into account that more than one atomic orbital can be relevant at each site. This leads to a richer and more accurate description of the system.


One can write the Hamiltonian by projecting the intercell and intracell hoppings to sites A and B: 
$$H_1^{ab}=\langle \psi_A | H_1 | \psi_B \rangle = \gamma _1  $$
$$H_2^{ab}=\langle \psi_A | H_2 | \psi_B \rangle = \gamma_2 $$
$$H = H_1 + H_2$$
Under such projection, the Hamiltonian is converted into a form of 2x2 matrix.

The Hamiltonian terms $H_1$ and $H_2$ are essentially representing the 'hopping' of an electron between adjacent sites in the lattice. $\gamma_1$ and $\gamma_2$ are known as the hopping integrals and their values depend on the overlap of the atomic orbitals on neighboring sites.

Under such projection, the Hamiltonian is converted into a form of 2x2 matrix, which can be solved to yield the energy levels of the system.

With the Bloch functions, the intercell hopping is then:
$$H_2^{ab}=\langle \psi_A | H_2 | \exp(-ika) \psi_B \rangle =\gamma_2 \exp(-ika)$$
considering the on-site potentials $\varepsilon_1$ and $\varepsilon_2$ on sites A and B, we write the Hamiltonian matrix in terms of wavevector k:
$$H_{00}=\left(\begin{array}{cc}
\varepsilon_1 & \gamma_1\\
\gamma_1 & \varepsilon_2
\end{array}\right)$$
$$H_{01}=\left(\begin{array}{cc}
0 & \gamma_2 \exp(ika)\\
0 & 0
\end{array}\right)$$
$$H_{10}=\left(\begin{array}{cc}
0 & 0\\
\gamma_2\exp(-ika) & 0
\end{array}\right)$$

The whole Hamiltonian matrix writes:
$$H(k)=H_{00}+H_{01}+H_{10}=\left(\begin{array}{cc}
\varepsilon_1 & \gamma_1+\gamma_2 \exp(ika)\\
\gamma_1+\gamma_2\exp(-ika) & \varepsilon_2
\end{array}\right)$$

In order to calculate the bands, one should diagonalize the Hamiltonian $H(k)$ for each $k$ in the first Brillouin zone, that is, to solve the following quadratic equation:
$$ (\lambda-\varepsilon_1)(\lambda-\varepsilon_2) = (\gamma_1 + \gamma_2\cos(ka))^2 + (\gamma_2\sin(ka))^2$$
<!---and the answer is :
#$$\lambda=\pm \sqrt{(\gamma_1 + \gamma_2\cos(ka))^2 + (\gamma_2\sin(ka))^2}$$--->



In this system, we consider the case where the two sites in the unit cell are not equivalent, leading to different on-site energies, $\varepsilon_1$ and $\varepsilon_2$. These could, for example, represent two different kinds of atoms in an alloy or two different atomic orbitals on the same atom.

By diagonalizing the Hamiltonian matrix, we can find the energies of the electronic states, and by doing this for all $k$-points in the Brillouin zone, we can map out the band structure of the material.

![image.png](image.png)   

In [3]:
nk = 40
k = np.linspace(-1.0 * np.pi, 1.0 * np.pi, nk)

p = figure(width=600, height=400)
r1 = p.line(k, k, color="cyan", line_width=2)
r2 = p.line(k, k, color="magenta", line_width=2)

def interactive_1dband(gamma_1, gamma_2, epsilon_1, epsilon_2):
    c = -(gamma_1 + gamma_2 * np.cos(k)) ** 2 - (gamma_2 * np.sin(k)) ** 2 + epsilon_1 * epsilon_2
    b = -(epsilon_1 + epsilon_2)
    a = 1
    
    x1 = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
    x2 = (-b - np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
    
    r1.data_source.data['y'] = x1
    r2.data_source.data['y'] = x2
    
    push_notebook()

interact(interactive_1dband, gamma_1=(0.5, 1.5), gamma_2=(0.5, 1.5), epsilon_1=(-0.5, 0.5), epsilon_2=(-0.5, 0.5))

show(p, notebook_handle=True)



interactive(children=(FloatSlider(value=1.0, description='gamma_1', max=1.5, min=0.5), FloatSlider(value=1.0, …

- How does the choice of $\gamma_1$ and $\gamma_2$ values influence the degree of hybridization between orbitals in the unit cell?

     - The degree of hybridization or mixing between orbitals is determined by the overlap between them, which in this model is quantified by the hopping parameters $\gamma_1$ and $\gamma_2$. Understanding how these parameters affect the band structure can provide valuable insights into the nature of the electronic states in the material.<br>
  
- How do the band structures change when you increase or decrease $\varepsilon_1$ and $\varepsilon_2$ while keeping $\gamma_1$ and $\gamma_2$ constant?
     
     - This question focuses on the influence of on-site energies on the band structure. On-site energies can be thought of as describing the energy required to add an electron to an atom or to remove an electron from it.<br>

- Discuss the physical implications of a band gap in the band structure. What does it mean for a material to have a large band gap versus a small or zero band gap?

    - Try to link the mathematical model to real physical properties.<br>

- What happens to the band structure if the symmetry between A and B sites is broken, i.e., if $\varepsilon_1 \neq \varepsilon_2$?
 
    - The concept of symmetry is crucial in understanding many properties of solids.<br>

- Can you explain how these ideas of band structure might extend to two or three dimensional systems? What additional complexity might arise in higher dimensions?



Remember, the goal of these exercises and questions is not just to manipulate equations and generate plots, but to develop an understanding of the physical meaning of these mathematical tools and how they relate to the properties of real materials. Always try to connect the mathematical results back to physical concepts.